In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import ShuffleSplit, GridSearchCV,train_test_split
from sklearn.model_selection import cross_validate

from sksurv.datasets import load_veterans_lung_cancer
from sksurv.column import encode_categorical
from sksurv.metrics import concordance_index_censored
from sksurv.svm import FastSurvivalSVM,FastKernelSurvivalSVM
from sksurv.kernels import clinical_kernel
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

##### In this notebook, we create the different features that we want to put in our evaluation model with a dictionary. After that,
##### we use a gridsearch on SVM survival to find a good set of parameters and we train our model 25 times with the best parameters found
##### using the training and validation set. Finally, we evaluate 25 times our model on a separate testing set.

In [35]:
def score_survival_model(model, X, y):
    prediction = model.predict(X)
    result = concordance_index_censored(y['Status'], y['Survival_in_days'], prediction)
    return result[0]

In [36]:
df_final = pd.read_table("../../../analysis/clustering/clustering_Final_1/df_final_full_component.tsv",sep=" ")

eln = [1,2,3]
comp =list(range(169,186)) 
#comp_overlap = list(range(167,197))
age = [166]

all_gen = list(range(4,88))
tmp = df_final.iloc[:,all_gen][df_final.iloc[:,all_gen] >0].count()
gen = [df_final.columns.get_loc(c) for c in tmp[tmp>df_final.shape[0]*0.02].keys() if c in df_final]

all_cyto = list(range(88,158))
tmp = df_final.iloc[:,all_cyto][df_final.iloc[:,all_cyto] >0].count()
cyto = [df_final.columns.get_loc(c) for c in tmp[tmp>df_final.shape[0]*0.02].keys() if c in df_final]

clin =list(range(158,165))
demo = [165,166]
demo_without_age = [165]

eln_comp = eln + comp
eln_gen = eln + gen
eln_cyto = eln + cyto
eln_clin = eln + clin
eln_demo = eln + demo

# USEFUL FOR ELN COMPARISON
# with comp
eln_comp_gen = eln_comp + gen
eln_comp_cyto = eln_comp + cyto
eln_comp_clin = eln_comp + clin
eln_comp_demo = eln_comp + demo




eln_comp_gen_cyto = eln_comp_gen + cyto
eln_comp_gen_clin = eln_comp_gen + clin
eln_comp_gen_demo = eln_comp_gen + demo

eln_comp_cyto_clin = eln_comp_cyto + clin
eln_comp_cyto_demo = eln_comp_cyto + demo


eln_comp_clin_demo = eln_comp_clin + demo


eln_comp_gen_cyto_clin_demo = eln_comp_gen_cyto + clin + demo
eln_comp_gen_cyto_clin_demo_without_age = eln_comp_gen_cyto + clin + demo_without_age
# without comp


eln_gen_cyto = eln_gen + cyto
eln_gen_clin = eln_gen + clin
eln_gen_demo = eln_gen + demo


eln_cyto_clin = eln_cyto + clin
eln_cyto_demo = eln_cyto + demo

eln_clin_demo = eln_clin + demo
eln_clin_demo_without_age = eln_clin + demo_without_age


eln_gen_cyto_clin_demo = eln_gen_cyto + clin + demo

# USEFUL FOR COMP

comp_gen = comp + gen
comp_cyto = comp + cyto
comp_clin = comp + clin
comp_demo = comp + demo
comp_gen_cyto = comp_gen + cyto
comp_clin_demo = comp_clin + demo
comp_gen_cyto_clin_demo = comp_gen_cyto + clin + demo

#USEFUL FOR GEN
gen_cyto = gen + cyto
gen_clin = gen + clin
gen_demo = gen + demo
gen_clin_demo = gen_clin + demo
gen_cyto_clin_demo = gen_cyto + clin + demo

#USEFUL FOR CYTO 
cyto_clin = cyto + clin
cyto_demo = cyto + demo
gen_demo_without_age = gen + demo_without_age
cyto_clin_demo = cyto_clin + demo
cyto_gen_demo = gen_cyto + demo


clin_demo  = clin + demo

dict_features_type_final_comp = dict(zip(("eln_comp","eln_comp_gen","eln","comp","gen","cyto","clin","demo","eln_gen","eln_cyto","eln_clin",
                          "eln_demo","eln_comp_cyto","eln_comp_clin","eln_comp_demo",
                          "eln_comp_gen_cyto","eln_comp_gen_clin","eln_comp_gen_demo",
                          "eln_comp_cyto_clin","eln_comp_cyto_demo","eln_comp_clin_demo",
                          "eln_comp_gen_cyto_clin_demo","eln_comp_gen_cyto_clin_demo_without_age",
                          "eln_gen_cyto","eln_clin_demo","eln_gen_cyto_clin_demo",
                          "comp_gen","comp_cyto","comp_clin","comp_demo","comp_gen_cyto","comp_clin_demo",
                          "comp_gen_cyto_clin_demo","gen_cyto","gen_clin","gen_demo","gen_clin_demo",
                          "gen_cyto_clin_demo","cyto_clin","cyto_demo","cyto_clin_demo","cyto_gen_demo","clin_demo"),(eln_comp,eln_comp_gen,eln,comp,gen,cyto,clin,demo,eln_gen,eln_cyto,eln_clin,
                          eln_demo,eln_comp_cyto,eln_comp_clin,eln_comp_demo,
                          eln_comp_gen_cyto,eln_comp_gen_clin,eln_comp_gen_demo,
                          eln_comp_cyto_clin,eln_comp_cyto_demo,eln_comp_clin_demo,
                          eln_comp_gen_cyto_clin_demo,eln_comp_gen_cyto_clin_demo_without_age,
                          eln_gen_cyto,eln_clin_demo,eln_gen_cyto_clin_demo,
                          comp_gen,comp_cyto,comp_clin,comp_demo,comp_gen_cyto,comp_clin_demo,
                          comp_gen_cyto_clin_demo,gen_cyto,gen_clin,gen_demo,gen_clin_demo,
                          gen_cyto_clin_demo,cyto_clin,cyto_demo,cyto_clin_demo,cyto_gen_demo,clin_demo)))
estimator = FastSurvivalSVM(max_iter=1000, tol=1e-6, random_state=17)
param_grid = {'alpha': 10. ** np.array([-6,-5,-4,-3,-2,-1,0]),'optimizer':["avltree"]}
cv = ShuffleSplit(n_splits=5,random_state=17)
gcv = GridSearchCV(estimator, param_grid, scoring=score_survival_model,
                   n_jobs=50, iid=False, refit=True,
                   cv=cv)
df=pd.DataFrame(columns=dict_features_type_final_comp.keys())
for key,item in dict_features_type_final_comp.items():
    x = df_final.iloc[:,item]
    y = np.array(list(zip(df_final.os_status, df_final.os)),dtype=[('Status', '?'), ('Survival_in_days', '<f8')])
    ci=[]
    for i in range(25):
        X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(x), y, test_size=0.2, random_state=i)
        gcv = gcv.fit(X_train,y_train)
        print(gcv.best_params_)
        ci.append(concordance_index_censored(y_test['Status'], y_test['Survival_in_days'], gcv.predict(X_test))[0])
        print(ci)
    df[key] = ci
    
df.to_csv("Prognosis_results_final/SVM.csv")

{'alpha': 0.1, 'optimizer': 'avltree'}
[0.6654315239462486]


/Users/taziy/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:740: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)


{'alpha': 0.0001, 'optimizer': 'avltree'}
[0.6654315239462486, 0.6315153346426953]
{'alpha': 0.1, 'optimizer': 'avltree'}
[0.6654315239462486, 0.6315153346426953, 0.6395260087191628]
{'alpha': 0.0001, 'optimizer': 'avltree'}
[0.6654315239462486, 0.6315153346426953, 0.6395260087191628, 0.6568636593749168]
{'alpha': 0.01, 'optimizer': 'avltree'}
[0.6654315239462486, 0.6315153346426953, 0.6395260087191628, 0.6568636593749168, 0.6403620901051313]
{'alpha': 0.001, 'optimizer': 'avltree'}
[0.6654315239462486, 0.6315153346426953, 0.6395260087191628, 0.6568636593749168, 0.6403620901051313, 0.6653246530752098]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.6654315239462486, 0.6315153346426953, 0.6395260087191628, 0.6568636593749168, 0.6403620901051313, 0.6653246530752098, 0.6303249048944093]
{'alpha': 0.01, 'optimizer': 'avltree'}
[0.6654315239462486, 0.6315153346426953, 0.6395260087191628, 0.6568636593749168, 0.6403620901051313, 0.6653246530752098, 0.6303249048944093, 0.6522217774219375]
{'alpha'

/Users/taziy/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:740: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)


{'alpha': 0.0001, 'optimizer': 'avltree'}
[0.6654315239462486, 0.6315153346426953, 0.6395260087191628, 0.6568636593749168, 0.6403620901051313, 0.6653246530752098, 0.6303249048944093, 0.6522217774219375, 0.6378225423460503, 0.6300593597353812, 0.6329601119664101, 0.6352318606161723, 0.6437368753301297, 0.6491456588246296, 0.6596264329776813]
{'alpha': 0.0001, 'optimizer': 'avltree'}
[0.6654315239462486, 0.6315153346426953, 0.6395260087191628, 0.6568636593749168, 0.6403620901051313, 0.6653246530752098, 0.6303249048944093, 0.6522217774219375, 0.6378225423460503, 0.6300593597353812, 0.6329601119664101, 0.6352318606161723, 0.6437368753301297, 0.6491456588246296, 0.6596264329776813, 0.6525657237936772]
{'alpha': 1.0, 'optimizer': 'avltree'}
[0.6654315239462486, 0.6315153346426953, 0.6395260087191628, 0.6568636593749168, 0.6403620901051313, 0.6653246530752098, 0.6303249048944093, 0.6522217774219375, 0.6378225423460503, 0.6300593597353812, 0.6329601119664101, 0.6352318606161723, 0.643736875330

...

{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.683887530878568, 0.7067807551264267, 0.6891760779291342, 0.7140363965788281, 0.6713145331698537, 0.7048800094885412, 0.7048702127945585, 0.690966106218308, 0.6953392960793625, 0.6875857382786958, 0.7005028121193272, 0.7002559955168944, 0.7028123832468017, 0.6932654669819512, 0.7153768764395965, 0.7033810316139767, 0.6770034072879548, 0.7130141068944963]
{'alpha': 0.0001, 'optimizer': 'avltree'}
[0.683887530878568, 0.7067807551264267, 0.6891760779291342, 0.7140363965788281, 0.6713145331698537, 0.7048800094885412, 0.7048702127945585, 0.690966106218308, 0.6953392960793625, 0.6875857382786958, 0.7005028121193272, 0.7002559955168944, 0.7028123832468017, 0.6932654669819512, 0.7153768764395965, 0.7033810316139767, 0.6770034072879548, 0.7130141068944963, 0.7154538674621778]
{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.683887530878568, 0.7067807551264267, 0.6891760779291342, 0.7140363965788281, 0.6713145331698537, 0.7048800094885412, 0.7048702127945585

/Users/taziy/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:740: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)


{'alpha': 0.0001, 'optimizer': 'avltree'}
[0.7000995874235311, 0.7109972668214094, 0.707041306063311, 0.7236684340944819, 0.6890277306759189, 0.7212477431768163, 0.7271712976032046, 0.711849479583667, 0.7123898475014511, 0.671059885383146, 0.711803125728948, 0.7110944126749621]
{'alpha': 0.0001, 'optimizer': 'avltree'}
[0.7000995874235311, 0.7109972668214094, 0.707041306063311, 0.7236684340944819, 0.6890277306759189, 0.7212477431768163, 0.7271712976032046, 0.711849479583667, 0.7123898475014511, 0.671059885383146, 0.711803125728948, 0.7110944126749621, 0.7308202677110576]
{'alpha': 0.0001, 'optimizer': 'avltree'}
[0.7000995874235311, 0.7109972668214094, 0.707041306063311, 0.7236684340944819, 0.6890277306759189, 0.7212477431768163, 0.7271712976032046, 0.711849479583667, 0.7123898475014511, 0.671059885383146, 0.711803125728948, 0.7110944126749621, 0.7308202677110576, 0.7077739035175611]
{'alpha': 1e-05, 'optimizer': 'avltree'}
[0.7000995874235311, 0.7109972668214094, 0.707041306063311, 0.

/Users/taziy/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:740: ConvergenceWarning: Optimization did not converge: Warning: Desired error not necessarily achieved due to precision loss.
  self.best_estimator_.fit(X, y, **fit_params)


{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.6723444431511013, 0.6705764397258581, 0.6937038330681362, 0.6971303722256268, 0.6687840763832729, 0.6884266153582583, 0.6909841244236534, 0.6896450493728316, 0.6951216294654635, 0.6578674085565199, 0.6922997174921598, 0.6938624756422176, 0.7043454735187642, 0.6761118458846005]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.6723444431511013, 0.6705764397258581, 0.6937038330681362, 0.6971303722256268, 0.6687840763832729, 0.6884266153582583, 0.6909841244236534, 0.6896450493728316, 0.6951216294654635, 0.6578674085565199, 0.6922997174921598, 0.6938624756422176, 0.7043454735187642, 0.6761118458846005, 0.7035821132614969]
{'alpha': 1e-06, 'optimizer': 'avltree'}
[0.6723444431511013, 0.6705764397258581, 0.6937038330681362, 0.6971303722256268, 0.6687840763832729, 0.6884266153582583, 0.6909841244236534, 0.6896450493728316, 0.6951216294654635, 0.6578674085565199, 0.6922997174921598, 0.6938624756422176, 0.7043454735187642, 0.6761118458846005, 0.703582113261

In [ ]:
df_final = pd.read_table("prognosis_dataframe.tsv",sep=" ")

eln = [1,2,3]
comp =[184]+list(range(197,213)) 
comp_overlap = list(range(167,197))
age = [166]

all_gen = list(range(4,88))
tmp = df_final.iloc[:,all_gen][df_final.iloc[:,all_gen] >0].count()
gen = [df_final.columns.get_loc(c) for c in tmp[tmp>df_final.shape[0]*0.02].keys() if c in df_final]

all_cyto = list(range(88,158))
tmp = df_final.iloc[:,all_cyto][df_final.iloc[:,all_cyto] >0].count()
cyto = [df_final.columns.get_loc(c) for c in tmp[tmp>df_final.shape[0]*0.02].keys() if c in df_final]

clin =list(range(158,165))
demo = [165,166]
demo_without_age = [165]

eln_comp = eln + comp
eln_gen = eln + gen
eln_cyto = eln + cyto
eln_clin = eln + clin
eln_demo = eln + demo

# USEFUL FOR ELN COMPARISON
# with comp
eln_comp_gen = eln_comp + gen
eln_comp_cyto = eln_comp + cyto
eln_comp_clin = eln_comp + clin
eln_comp_demo = eln_comp + demo




eln_comp_gen_cyto = eln_comp_gen + cyto
eln_comp_gen_clin = eln_comp_gen + clin
eln_comp_gen_demo = eln_comp_gen + demo

eln_comp_cyto_clin = eln_comp_cyto + clin
eln_comp_cyto_demo = eln_comp_cyto + demo


eln_comp_clin_demo = eln_comp_clin + demo


eln_comp_gen_cyto_clin_demo = eln_comp_gen_cyto + clin + demo
eln_comp_gen_cyto_clin_demo_without_age = eln_comp_gen_cyto + clin + demo_without_age
# without comp


eln_gen_cyto = eln_gen + cyto
eln_gen_clin = eln_gen + clin
eln_gen_demo = eln_gen + demo


eln_cyto_clin = eln_cyto + clin
eln_cyto_demo = eln_cyto + demo

eln_clin_demo = eln_clin + demo
eln_clin_demo_without_age = eln_clin + demo_without_age


eln_gen_cyto_clin_demo = eln_gen_cyto + clin + demo

# USEFUL FOR COMP

comp_gen = comp + gen
comp_cyto = comp + cyto
comp_clin = comp + clin
comp_demo = comp + demo
comp_gen_cyto = comp_gen + cyto
comp_clin_demo = comp_clin + demo
comp_gen_cyto_clin_demo = comp_gen_cyto + clin + demo

#USEFUL FOR GEN
gen_cyto = gen + cyto
gen_clin = gen + clin
gen_demo = gen + demo
gen_clin_demo = gen_clin + demo
gen_cyto_clin_demo = gen_cyto + clin + demo

#USEFUL FOR CYTO 
cyto_clin = cyto + clin
cyto_demo = cyto + demo
gen_demo_without_age = gen + demo_without_age
cyto_clin_demo = cyto_clin + demo
cyto_gen_demo = gen_cyto + demo


clin_demo  = clin + demo

dict_features_type_final_comp = dict(zip(("eln_comp","eln_comp_gen","eln","comp","gen","cyto","clin","demo","eln_gen","eln_cyto","eln_clin",
                          "eln_demo","eln_comp_cyto","eln_comp_clin","eln_comp_demo",
                          "eln_comp_gen_cyto","eln_comp_gen_clin","eln_comp_gen_demo",
                          "eln_comp_cyto_clin","eln_comp_cyto_demo","eln_comp_clin_demo",
                          "eln_comp_gen_cyto_clin_demo","eln_comp_gen_cyto_clin_demo_without_age",
                          "eln_gen_cyto","eln_clin_demo","eln_gen_cyto_clin_demo",
                          "comp_gen","comp_cyto","comp_clin","comp_demo","comp_gen_cyto","comp_clin_demo",
                          "comp_gen_cyto_clin_demo","gen_cyto","gen_clin","gen_demo","gen_clin_demo",
                          "gen_cyto_clin_demo","cyto_clin","cyto_demo","cyto_clin_demo","cyto_gen_demo","clin_demo"),(eln_comp,eln_comp_gen,eln,comp,gen,cyto,clin,demo,eln_gen,eln_cyto,eln_clin,
                          eln_demo,eln_comp_cyto,eln_comp_clin,eln_comp_demo,
                          eln_comp_gen_cyto,eln_comp_gen_clin,eln_comp_gen_demo,
                          eln_comp_cyto_clin,eln_comp_cyto_demo,eln_comp_clin_demo,
                          eln_comp_gen_cyto_clin_demo,eln_comp_gen_cyto_clin_demo_without_age,
                          eln_gen_cyto,eln_clin_demo,eln_gen_cyto_clin_demo,
                          comp_gen,comp_cyto,comp_clin,comp_demo,comp_gen_cyto,comp_clin_demo,
                          comp_gen_cyto_clin_demo,gen_cyto,gen_clin,gen_demo,gen_clin_demo,
                          gen_cyto_clin_demo,cyto_clin,cyto_demo,cyto_clin_demo,cyto_gen_demo,clin_demo)))
estimator = FastSurvivalSVM(max_iter=1000, tol=1e-6, random_state=17)
param_grid = {'alpha': 10. ** np.array([-6,-5,-4,-3,-2,-1,0]),'optimizer':["avltree"]}
cv = ShuffleSplit(n_splits=5,random_state=17)
gcv = GridSearchCV(estimator, param_grid, scoring=score_survival_model,
                   n_jobs=50, iid=False, refit=True,
                   cv=cv)
df=pd.DataFrame(columns=dict_features_type_final_comp.keys())
for key,item in dict_features_type_final_comp.items():
    x = df_final.iloc[:,item]
    y = np.array(list(zip(df_final.os_status, df_final.os)),dtype=[('Status', '?'), ('Survival_in_days', '<f8')])
    ci=[]
    for i in range(25):
        X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(x), y, test_size=0.2, random_state=i)
        gcv = gcv.fit(X_train,y_train)
        print(gcv.best_params_)
        ci.append(concordance_index_censored(y_test['Status'], y_test['Survival_in_days'], gcv.predict(X_test))[0])
        print(ci)
    df[key] = ci
    
df.to_csv("Prognosis_results/SVM.csv")

In [ ]:
df_final = pd.read_table("prognosis_dataframe.tsv",sep=" ")

eln = [1,2,3]
comp = list(range(167,197))
age = [166]

all_gen = list(range(4,88))
tmp = df_final.iloc[:,all_gen][df_final.iloc[:,all_gen] >0].count()
gen = [df_final.columns.get_loc(c) for c in tmp[tmp>df_final.shape[0]*0.02].keys() if c in df_final]

all_cyto = list(range(88,158))
tmp = df_final.iloc[:,all_cyto][df_final.iloc[:,all_cyto] >0].count()
cyto = [df_final.columns.get_loc(c) for c in tmp[tmp>df_final.shape[0]*0.02].keys() if c in df_final]

clin =list(range(158,165))
demo = [165,166]
demo_without_age = [165]

eln_comp = eln + comp
eln_gen = eln + gen
eln_cyto = eln + cyto
eln_clin = eln + clin
eln_demo = eln + demo

# USEFUL FOR ELN COMPARISON
# with comp
eln_comp_gen = eln_comp + gen
eln_comp_cyto = eln_comp + cyto
eln_comp_clin = eln_comp + clin
eln_comp_demo = eln_comp + demo




eln_comp_gen_cyto = eln_comp_gen + cyto
eln_comp_gen_clin = eln_comp_gen + clin
eln_comp_gen_demo = eln_comp_gen + demo

eln_comp_cyto_clin = eln_comp_cyto + clin
eln_comp_cyto_demo = eln_comp_cyto + demo


eln_comp_clin_demo = eln_comp_clin + demo


eln_comp_gen_cyto_clin_demo = eln_comp_gen_cyto + clin + demo
eln_comp_gen_cyto_clin_demo_without_age = eln_comp_gen_cyto + clin + demo_without_age
# without comp


eln_gen_cyto = eln_gen + cyto
eln_gen_clin = eln_gen + clin
eln_gen_demo = eln_gen + demo


eln_cyto_clin = eln_cyto + clin
eln_cyto_demo = eln_cyto + demo

eln_clin_demo = eln_clin + demo
eln_clin_demo_without_age = eln_clin + demo_without_age


eln_gen_cyto_clin_demo = eln_gen_cyto + clin + demo

# USEFUL FOR COMP

comp_gen = comp + gen
comp_cyto = comp + cyto
comp_clin = comp + clin
comp_demo = comp + demo
comp_gen_cyto = comp_gen + cyto
comp_clin_demo = comp_clin + demo
comp_gen_cyto_clin_demo = comp_gen_cyto + clin + demo

#USEFUL FOR GEN
gen_cyto = gen + cyto
gen_clin = gen + clin
gen_demo = gen + demo
gen_clin_demo = gen_clin + demo
gen_cyto_clin_demo = gen_cyto + clin + demo

#USEFUL FOR CYTO 
cyto_clin = cyto + clin
cyto_demo = cyto + demo
gen_demo_without_age = gen + demo_without_age
cyto_clin_demo = cyto_clin + demo
cyto_gen_demo = gen_cyto + demo


clin_demo  = clin + demo

dict_features_type_final_comp = dict(zip(("eln_comp","eln_comp_gen","eln","comp","gen","cyto","clin","demo","eln_gen","eln_cyto","eln_clin",
                          "eln_demo","eln_comp_cyto","eln_comp_clin","eln_comp_demo",
                          "eln_comp_gen_cyto","eln_comp_gen_clin","eln_comp_gen_demo",
                          "eln_comp_cyto_clin","eln_comp_cyto_demo","eln_comp_clin_demo",
                          "eln_comp_gen_cyto_clin_demo","eln_comp_gen_cyto_clin_demo_without_age",
                          "eln_gen_cyto","eln_clin_demo","eln_gen_cyto_clin_demo",
                          "comp_gen","comp_cyto","comp_clin","comp_demo","comp_gen_cyto","comp_clin_demo",
                          "comp_gen_cyto_clin_demo","gen_cyto","gen_clin","gen_demo","gen_clin_demo",
                          "gen_cyto_clin_demo","cyto_clin","cyto_demo","cyto_clin_demo","cyto_gen_demo","clin_demo"),(eln_comp,eln_comp_gen,eln,comp,gen,cyto,clin,demo,eln_gen,eln_cyto,eln_clin,
                          eln_demo,eln_comp_cyto,eln_comp_clin,eln_comp_demo,
                          eln_comp_gen_cyto,eln_comp_gen_clin,eln_comp_gen_demo,
                          eln_comp_cyto_clin,eln_comp_cyto_demo,eln_comp_clin_demo,
                          eln_comp_gen_cyto_clin_demo,eln_comp_gen_cyto_clin_demo_without_age,
                          eln_gen_cyto,eln_clin_demo,eln_gen_cyto_clin_demo,
                          comp_gen,comp_cyto,comp_clin,comp_demo,comp_gen_cyto,comp_clin_demo,
                          comp_gen_cyto_clin_demo,gen_cyto,gen_clin,gen_demo,gen_clin_demo,
                          gen_cyto_clin_demo,cyto_clin,cyto_demo,cyto_clin_demo,cyto_gen_demo,clin_demo)))
estimator = FastSurvivalSVM(max_iter=1000, tol=1e-6, random_state=17)
param_grid = {'alpha': 10. ** np.array([-6,-5,-4,-3,-2,-1,0]),'optimizer':["avltree"]}
cv = ShuffleSplit(n_splits=5,random_state=17)
gcv = GridSearchCV(estimator, param_grid, scoring=score_survival_model,
                   n_jobs=50, iid=False, refit=True,
                   cv=cv)
df=pd.DataFrame(columns=dict_features_type_final_comp.keys())
for key,item in dict_features_type_final_comp.items():
    x = df_final.iloc[:,item]
    y = np.array(list(zip(df_final.os_status, df_final.os)),dtype=[('Status', '?'), ('Survival_in_days', '<f8')])
    ci=[]
    for i in range(25):
        X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(x), y, test_size=0.2, random_state=i)
        gcv = gcv.fit(X_train,y_train)
        print(gcv.best_params_)
        ci.append(concordance_index_censored(y_test['Status'], y_test['Survival_in_days'], gcv.predict(X_test))[0])
        print(ci)
    df[key] = ci
    
df.to_csv("Prognosis_results_overlap/SVM_overlap.csv")

In [34]:
pd.read_csv("SVM.csv").mean(0).sum()

41.05433236414501

In [33]:
pd.read_csv("SVM_overlap.csv").mean(0).sum()

41.060044528081605